<h1> <center> Detecting Fraudulent Credit Card Transactions </h1> </center>

## Introduction
It is important that credit card companies are able to recognize fraudulent credit card transactions so that customers are not charged for items that they did not purchase  [$^1$](https://www.kaggle.com/mlg-ulb/creditcardfraud). 


## Data
The [dataset](https://www.kaggle.com/mlg-ulb/creditcardfraud) contains transactions made by credit cards in September 2013 by European card-holders. This dataset presents transactions that occurred in two days, where there are 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, the original features and more background information about the data is not provided. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise [$^1$](https://www.kaggle.com/mlg-ulb/creditcardfraud).

Given the class imbalance ratio, the classification accuracy is measured using the average classification precision. Confusion matrix accuracy is not meaningful for unbalanced classification.

## Problem Statement
Classify the transaction as fraud/not fraud using the 28 PCA transformed features provided.

## Approach
Data imbalance was handled using:
- Voting
- Upsampling
- Downsampling

After each imbalance handling method was applied, data was classified using logistic regression and neural network to compare accuracy of the classification problem.

It is found that all 3 imbalance handling methods work equally well. 

## Conclusion 
Maximum average precision of 0.96 was achieved using neural networks and oversampling.

## Importing Libraries

In [36]:
import csv
import numpy as np

from sklearn import svm

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc
from sklearn.model_selection import train_test_split 

from imblearn.over_sampling import SMOTE 

## Reading data

In [2]:
with open('creditcard.csv','rt')as f:
    data1 = csv.reader(f)
    datax =[]
    for row in data1:
        datax.append(row)
datax = np.array(datax)
columns = datax[0]
data = np.delete(datax,0,axis = 0)

In [3]:
print("data dimensions: ", data.shape)

data dimensions:  (284807, 31)


In [4]:
labels = data[:,30]
x = data[:,:30]
print("x dimensions: ",x.shape)
print("labels: ",labels.shape)
print("x data type: ",type(x[0][0]))

x dimensions:  (284807, 30)
labels:  (284807,)
x data type:  <class 'numpy.str_'>


### Method 1 - train k models and apply voting

#### Partitioning data by class

In [5]:
data0 = []
data1 = []
for i in np.arange(x.shape[0]):
    temp = x[i].astype(np.float)
#     v= labels[i].astype(np.float)
    if labels[i]==str(0):
        data0.append(temp)
    else:
        data1.append(temp)
data0x = np.array(data0)
data1x = np.array(data1)
print("data0x shape",data0x.shape)
print("data1x shape",data1x.shape)

data0x shape (284315, 30)
data1x shape (492, 30)


#### Converting class imbalance to 90:10

In [6]:
p = data1x.shape[0]*9
data0 = data0x[:p,:]
data1 = data1x
print(data0.shape)
print(data1.shape)

(4428, 30)
(492, 30)


#### Splitting into train and test sets (50)

In [7]:
#Doing Ensemble(Taking 492 samples from data0 and making 9 training sets out of it. Then Training 9 
#classifiers since ratio is 9:1)
test_data0 = data0[:50,:]
test_data1 = data1[:50,:]
print("test_data0 shape",test_data0.shape)
print("test_data1 shape",test_data1.shape)

train_data0 = data0[50:,:]
train_data1 = data1[50:,:]
print("train_data0 shape",train_data0.shape)
print("train_data1 shape",train_data1.shape)

test_data0 shape (50, 30)
test_data1 shape (50, 30)
train_data0 shape (4378, 30)
train_data1 shape (442, 30)


#### Preparing training data (0) into k sets

In [8]:
split_data0 = []
n1train=train_data1.shape[0]
n0train=train_data0.shape[0]
k = int(n0train/n1train)
c = 0
for i in np.arange(k):
    split_data0.append(train_data0[c*n1train:(c+1)*n1train][:])
    c=c+1
for i in np.arange(k):
    print("shape of",i,": ",split_data0[i].shape)

shape of 0 :  (442, 30)
shape of 1 :  (442, 30)
shape of 2 :  (442, 30)
shape of 3 :  (442, 30)
shape of 4 :  (442, 30)
shape of 5 :  (442, 30)
shape of 6 :  (442, 30)
shape of 7 :  (442, 30)
shape of 8 :  (442, 30)


#### Preparing testing data

In [9]:
label0 = np.zeros((test_data0.shape[0]))
label1 = np.ones((test_data1.shape[0]))
test_data = np.concatenate((test_data0,test_data1),axis = 0)
test_label = np.concatenate((label0,label1),axis = 0)

#### Binary classification - training

In [10]:
model = Sequential()
model.add(Dense(15, input_dim=30, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

models = []
for i in np.arange(k):
    #creating training dataset and labels for each model run
    label0 = np.zeros((split_data0[i].shape[0],1))
    label1 = np.ones((train_data1.shape[0],1))
    x_train = np.concatenate((split_data0[i],train_data1),axis = 0)
    y_train = np.concatenate((label0,label1),axis = 0)
    dataset = np.concatenate((x_train,y_train),axis = 1)
    np.random.shuffle(dataset)
    y_train = dataset[:,30]
    x_train = dataset[:,:30]
    history = model.fit(x_train, y_train, epochs=100, batch_size=64)
    models.append(model)
    print("Completed Model",i+1)

Train on 884 samples
Epoch 1/100
884/884 [==============================] - 1s 1ms/sample - loss: 5080.8407 - accuracy: 0.4796
Epoch 2/100
884/884 [==============================] - 0s 26us/sample - loss: 2772.9975 - accuracy: 0.4661
Epoch 3/100
884/884 [==============================] - 0s 26us/sample - loss: 601.8323 - accuracy: 0.3258
Epoch 4/100
884/884 [==============================] - 0s 25us/sample - loss: 4.9410 - accuracy: 0.5011
Epoch 5/100
884/884 [==============================] - 0s 27us/sample - loss: 5.0428 - accuracy: 0.5034
Epoch 6/100
884/884 [==============================] - 0s 23us/sample - loss: 3.7599 - accuracy: 0.5396
Epoch 7/100
884/884 [==============================] - 0s 24us/sample - loss: 2.0169 - accuracy: 0.5939
Epoch 8/100
884/884 [==============================] - 0s 28us/sample - loss: 2.8788 - accuracy: 0.6210
Epoch 9/100
884/884 [==============================] - 0s 29us/sample - loss: 1.2195 - accuracy: 0.6278
Epoch 10/100
884/884 [==============

884/884 [==============================] - 0s 37us/sample - loss: 0.0109 - accuracy: 1.0000
Epoch 56/100
884/884 [==============================] - 0s 51us/sample - loss: 0.0118 - accuracy: 1.0000
Epoch 57/100
884/884 [==============================] - 0s 25us/sample - loss: 0.0107 - accuracy: 1.0000
Epoch 58/100
884/884 [==============================] - 0s 25us/sample - loss: 0.0120 - accuracy: 0.9989
Epoch 59/100
884/884 [==============================] - 0s 24us/sample - loss: 0.0133 - accuracy: 1.0000
Epoch 60/100
884/884 [==============================] - 0s 24us/sample - loss: 0.0128 - accuracy: 1.0000
Epoch 61/100
884/884 [==============================] - 0s 25us/sample - loss: 0.0108 - accuracy: 1.0000
Epoch 62/100
884/884 [==============================] - 0s 44us/sample - loss: 0.0092 - accuracy: 1.0000
Epoch 63/100
884/884 [==============================] - 0s 29us/sample - loss: 0.0097 - accuracy: 0.9989
Epoch 64/100
884/884 [==============================] - 0s 28us/samp

884/884 [==============================] - 0s 55us/sample - loss: 0.0050 - accuracy: 1.0000
Epoch 10/100
884/884 [==============================] - 0s 33us/sample - loss: 0.0046 - accuracy: 1.0000
Epoch 11/100
884/884 [==============================] - 0s 37us/sample - loss: 0.0066 - accuracy: 1.0000
Epoch 12/100
884/884 [==============================] - 0s 33us/sample - loss: 0.0048 - accuracy: 1.0000
Epoch 13/100
884/884 [==============================] - 0s 30us/sample - loss: 0.0032 - accuracy: 1.0000
Epoch 14/100
884/884 [==============================] - 0s 27us/sample - loss: 0.0035 - accuracy: 1.0000
Epoch 15/100
884/884 [==============================] - 0s 28us/sample - loss: 0.0039 - accuracy: 1.0000
Epoch 16/100
884/884 [==============================] - 0s 29us/sample - loss: 0.0228 - accuracy: 0.9989
Epoch 17/100
884/884 [==============================] - 0s 26us/sample - loss: 0.0058 - accuracy: 1.0000
Epoch 18/100
884/884 [==============================] - 0s 24us/samp

884/884 [==============================] - 0s 29us/sample - loss: 6.0876e-04 - accuracy: 1.0000
Epoch 61/100
884/884 [==============================] - 0s 25us/sample - loss: 6.5123e-04 - accuracy: 1.0000
Epoch 62/100
884/884 [==============================] - 0s 30us/sample - loss: 6.1057e-04 - accuracy: 1.0000
Epoch 63/100
884/884 [==============================] - 0s 26us/sample - loss: 5.9684e-04 - accuracy: 1.0000
Epoch 64/100
884/884 [==============================] - 0s 24us/sample - loss: 6.0471e-04 - accuracy: 1.0000
Epoch 65/100
884/884 [==============================] - 0s 24us/sample - loss: 5.8656e-04 - accuracy: 1.0000
Epoch 66/100
884/884 [==============================] - 0s 25us/sample - loss: 0.0012 - accuracy: 1.0000
Epoch 67/100
884/884 [==============================] - 0s 23us/sample - loss: 0.0011 - accuracy: 1.0000
Epoch 68/100
884/884 [==============================] - 0s 25us/sample - loss: 8.0648e-04 - accuracy: 1.0000
Epoch 69/100
884/884 [==================

884/884 [==============================] - 0s 27us/sample - loss: 5.2048 - accuracy: 0.7251
Epoch 12/100
884/884 [==============================] - 0s 26us/sample - loss: 3.3959 - accuracy: 0.8077
Epoch 13/100
884/884 [==============================] - 0s 28us/sample - loss: 1.9299 - accuracy: 0.7602
Epoch 14/100
884/884 [==============================] - 0s 26us/sample - loss: 1.4900 - accuracy: 0.8563
Epoch 15/100
884/884 [==============================] - 0s 28us/sample - loss: 0.9169 - accuracy: 0.8541
Epoch 16/100
884/884 [==============================] - 0s 36us/sample - loss: 0.9080 - accuracy: 0.8303
Epoch 17/100
884/884 [==============================] - 0s 37us/sample - loss: 0.5720 - accuracy: 0.8869
Epoch 18/100
884/884 [==============================] - 0s 39us/sample - loss: 0.3998 - accuracy: 0.8971
Epoch 19/100
884/884 [==============================] - 0s 28us/sample - loss: 0.2602 - accuracy: 0.9310
Epoch 20/100
884/884 [==============================] - 0s 33us/samp

Epoch 89/100
884/884 [==============================] - 0s 29us/sample - loss: 8.8725e-04 - accuracy: 1.0000
Epoch 90/100
884/884 [==============================] - 0s 30us/sample - loss: 0.0010 - accuracy: 1.0000
Epoch 91/100
884/884 [==============================] - 0s 28us/sample - loss: 0.0012 - accuracy: 1.0000
Epoch 92/100
884/884 [==============================] - 0s 27us/sample - loss: 0.0010 - accuracy: 1.0000
Epoch 93/100
884/884 [==============================] - 0s 28us/sample - loss: 8.1717e-04 - accuracy: 1.0000
Epoch 94/100
884/884 [==============================] - 0s 29us/sample - loss: 8.7686e-04 - accuracy: 1.0000
Epoch 95/100
884/884 [==============================] - 0s 25us/sample - loss: 9.7610e-04 - accuracy: 1.0000
Epoch 96/100
884/884 [==============================] - 0s 26us/sample - loss: 8.6025e-04 - accuracy: 1.0000
Epoch 97/100
884/884 [==============================] - 0s 37us/sample - loss: 7.8951e-04 - accuracy: 1.0000
Epoch 98/100
884/884 [=========

884/884 [==============================] - 0s 41us/sample - loss: 0.0108 - accuracy: 0.9943
Epoch 42/100
884/884 [==============================] - 0s 36us/sample - loss: 0.0073 - accuracy: 0.9966
Epoch 43/100
884/884 [==============================] - 0s 55us/sample - loss: 0.0025 - accuracy: 0.9989
Epoch 44/100
884/884 [==============================] - 0s 42us/sample - loss: 0.0101 - accuracy: 0.9943
Epoch 45/100
884/884 [==============================] - 0s 45us/sample - loss: 0.0131 - accuracy: 0.9932
Epoch 46/100
884/884 [==============================] - 0s 44us/sample - loss: 0.0040 - accuracy: 0.9966
Epoch 47/100
884/884 [==============================] - 0s 42us/sample - loss: 0.0016 - accuracy: 1.0000
Epoch 48/100
884/884 [==============================] - 0s 83us/sample - loss: 0.0017 - accuracy: 1.0000
Epoch 49/100
884/884 [==============================] - 0s 35us/sample - loss: 0.0018 - accuracy: 1.0000
Epoch 50/100
884/884 [==============================] - 0s 46us/samp

#### Testing

In [11]:
predict = []
for j in np.arange(k):
    p = models[j].predict(test_data)
    predict.append(p)
predict = np.array(predict)
prediction = np.transpose(predict)
# print(prediction.shape)
prediction = prediction[0]
print(prediction.shape)
prediction=1*(prediction>=0.5)

result = []
for i in np.arange(prediction.shape[0]):
    c=0
    for j in np.arange(k):
        if prediction[i][j]==0:
            c=c+1
    if c>k-c:
        result.append(0)
    else:
        result.append(1)
result = np.array(result)

(100, 9)


#### Metrics

In [12]:
tp = 0
fp = 0
tn = 0
fn = 0
for i in np.arange(result.shape[0]):
    if int(result[i]) == 0 and int(test_label[i])==int(result[i]):
        tp = tp + 1
    if int(result[i]) == 0 and int(test_label[i])!=int(result[i]):
        fp = fp + 1
    if int(result[i]) == 1 and int(test_label[i])==int(result[i]):
        tn = tn + 1
    if int(result[i]) == 1 and int(test_label[i])!=int(result[i]):
        fn = fn + 1
if tp + fp !=0:
    print("Precision:", tp/(tp+fp))
else:
    print("Precision:",np.inf)
if tp + fn !=0:
    print("Recall:", tp/(tp+fn))
else:
    print("Recall:",np.inf)
        

Precision: 0.9615384615384616
Recall: 1.0


### Oversampling & Downsampling

In [14]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
from sklearn.linear_model import LogisticRegression 
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import confusion_matrix, classification_report 

import csv
import numpy as np

In [15]:
#Applying SMOTE Algorithm
#Link: https://www.geeksforgeeks.org/ml-handling-imbalanced-data-with-smote-and-near-miss-algorithm-in-python/


In [16]:
with open('creditcard.csv','rt')as f:
    data1 = csv.reader(f)
    datax =[]
    for row in data1:
        datax.append(row)
datax = np.array(datax)

In [17]:
datax.shape

(284808, 31)

In [18]:
datax[0]

array(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9',
       'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18',
       'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27',
       'V28', 'Amount', 'Class'], dtype='<U21')

In [19]:
datax = np.delete(datax,0,axis = 0)

In [20]:
datax.shape

(284807, 31)

In [21]:
datax = datax.astype(np.float)

In [22]:
X = datax[:,:30]
y = datax[:,30]
y = y.astype(np.int)

In [23]:
X.shape

(284807, 30)

In [24]:
# split into 70:30 ration 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0) 

# describes info about train and test set 
print("Number of transactions in X_train dataset: ", X_train.shape) 
print("Number of transactions in y_train dataset: ", y_train.shape) 
print("Number of transactions in X_test dataset: ", X_test.shape) 
print("Number of transactions in y_test dataset: ", y_test.shape) 

Number of transactions in X_train dataset:  (199364, 30)
Number of transactions in y_train dataset:  (199364,)
Number of transactions in X_test dataset:  (85443, 30)
Number of transactions in y_test dataset:  (85443,)


#### Method 2 - Oversampling

In [25]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1))) 
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0))) 

Before OverSampling, counts of label '1': 345
Before OverSampling, counts of label '0': 199019 



In [26]:
sm = SMOTE(random_state = 2) 
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel()) 

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape)) 
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape)) 

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1))) 
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0))) 

C:\Users\Aneri\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


After OverSampling, the shape of train_X: (398038, 30)
After OverSampling, the shape of train_y: (398038,) 

After OverSampling, counts of label '1': 199019
After OverSampling, counts of label '0': 199019


##### Logistic regression

In [27]:
lr1 = LogisticRegression() 
lr1.fit(X_train_res, y_train_res.ravel()) 
predictions = lr1.predict(X_test) 

# print classification report 
print(classification_report(y_test, predictions)) 

              precision    recall  f1-score   support

           0       1.00      0.98      0.99     85296
           1       0.08      0.87      0.14       147

    accuracy                           0.98     85443
   macro avg       0.54      0.93      0.56     85443
weighted avg       1.00      0.98      0.99     85443



##### Neural Network

In [28]:
#Using Neural Network defined for first part
model.fit(X_train_res, y_train_res.ravel(),epochs=10, batch_size=64) 
predictions = model.predict(X_test) 
print(predictions.shape)

Train on 398038 samples
Epoch 1/10
398038/398038 [==============================] - 11s 28us/sample - loss: 5.0239 - accuracy: 0.8973
Epoch 2/10
398038/398038 [==============================] - 8s 21us/sample - loss: 2.9131 - accuracy: 0.9360
Epoch 3/10
398038/398038 [==============================] - 9s 24us/sample - loss: 2.4560 - accuracy: 0.9480
Epoch 4/10
398038/398038 [==============================] - 14s 35us/sample - loss: 2.4096 - accuracy: 0.9518
Epoch 5/10
398038/398038 [==============================] - 9s 24us/sample - loss: 2.3348 - accuracy: 0.9549
Epoch 6/10
398038/398038 [==============================] - 11s 28us/sample - loss: 2.3065 - accuracy: 0.9566
Epoch 7/10
398038/398038 [==============================] - 15s 37us/sample - loss: 2.0492 - accuracy: 0.9594
Epoch 8/10
398038/398038 [==============================] - 11s 27us/sample - loss: 1.9092 - accuracy: 0.9608
Epoch 9/10
398038/398038 [==============================] - 8s 21us/sample - loss: 1.9331 - accurac

In [29]:
result = []
predictions=1*(predictions<=0.5)
# for i in np.arange(predictions.shape[0]):
#     if predictions[i]>0.5:
#         result.append(0)
#     else:
#         result.append(1)
# result = np.array(result)

In [30]:
# print classification report 
print(classification_report(y_test, predictions)) 

              precision    recall  f1-score   support

           0       0.92      0.02      0.03     85296
           1       0.00      0.12      0.00       147

    accuracy                           0.02     85443
   macro avg       0.46      0.07      0.02     85443
weighted avg       0.91      0.02      0.03     85443



#### Method 3 - Downsampling

In [31]:
# # print("data0 shape:",data0.shape)
# # print("data1 shape:",data1.shape)
# print("ratio:",int(data0/data1))
# test_data = np.concatenate((data0[:50,:],data1[:50,:]),axis = 0)
# test_label = np.concatenate((np.zeros((50,1)),np.ones((50,1))),axis = 0)
# print("test data",test_data.shape)
# print("test label",test_label.shape)
# train_data0 = data0[50:,:]
# train_data1 = data1[50:,:]
# print("train_data0 shape",train_data0.shape)
# print("train_data1 shape",train_data1.shape)

##### Neural Network

In [32]:
#Using Downsampling
#Initially the ratio of size is 10:90. We will decrease the training set sizes
#and see how the average accuracy changes
avg_prec = []
for i in np.arange(9):
    #preparing training data
    size0 = (i+1)*train_data1.shape[0]
    size1 = train_data1.shape[0]
    np.random.shuffle(train_data0)
    np.random.shuffle(train_data1)
    train_data = np.concatenate((train_data0[:size0,:],train_data1[:size1]),axis = 0)
    train_label = np.concatenate((np.zeros((size0,1)),np.ones((size1,1))),axis = 0)
    data = np.concatenate((train_data,train_label),axis=1)
    
    np.random.shuffle(data)
    y_train = data[:,data.shape[1]-1]
    x_train = np.delete(data,data.shape[1]-1,axis=1)
    
    #calculating average accuracy over 5 runs for each training size and ratio
    avg = 0
    for k in np.arange(5):
        history = model.fit(x_train,y_train,epochs=10,batch_size=30)
        predict = model.predict(test_data)
        print("predict shape:",predict.shape)
        result = []
        for l in np.arange(predict.shape[0]):
            if predict[l]<0.5:
                result.append(0)
            else:
                result.append(1)
        result = np.array(result)
        tn, fp, fn, tp = confusion_matrix(test_label,result).ravel()#Confusion Matrix takes 0 as negative but in our case 0 is positive class
        avg = avg + tn/(tn+fn)
    avg_prec.append(avg/5)

Train on 884 samples
Epoch 1/10
884/884 [==============================] - 0s 47us/sample - loss: 1.2347 - accuracy: 0.9276
Epoch 2/10
884/884 [==============================] - 0s 45us/sample - loss: 1.1522 - accuracy: 0.9491
Epoch 3/10
884/884 [==============================] - 0s 45us/sample - loss: 2.5249 - accuracy: 0.8654
Epoch 4/10
884/884 [==============================] - 0s 55us/sample - loss: 2.2076 - accuracy: 0.8597
Epoch 5/10
884/884 [==============================] - 0s 52us/sample - loss: 0.9858 - accuracy: 0.9344
Epoch 6/10
884/884 [==============================] - 0s 45us/sample - loss: 0.6825 - accuracy: 0.9548
Epoch 7/10
884/884 [==============================] - 0s 61us/sample - loss: 0.6363 - accuracy: 0.9627
Epoch 8/10
884/884 [==============================] - 0s 73us/sample - loss: 0.5714 - accuracy: 0.9604
Epoch 9/10
884/884 [==============================] - 0s 52us/sample - loss: 0.4487 - accuracy: 0.9638
Epoch 10/10
884/884 [==============================]

1768/1768 [==============================] - 0s 51us/sample - loss: 2.1991e-05 - accuracy: 1.0000
Epoch 10/10
1768/1768 [==============================] - 0s 46us/sample - loss: 2.2098e-05 - accuracy: 1.0000
predict shape: (100, 1)
Train on 2210 samples
Epoch 1/10
2210/2210 [==============================] - 0s 44us/sample - loss: 0.0617 - accuracy: 0.9932
Epoch 2/10
2210/2210 [==============================] - 0s 44us/sample - loss: 0.8553 - accuracy: 0.9443
Epoch 3/10
2210/2210 [==============================] - 0s 58us/sample - loss: 3.0830 - accuracy: 0.9090
Epoch 4/10
2210/2210 [==============================] - 0s 55us/sample - loss: 0.8469 - accuracy: 0.9529
Epoch 5/10
2210/2210 [==============================] - 0s 51us/sample - loss: 0.2533 - accuracy: 0.9792
Epoch 6/10
2210/2210 [==============================] - 0s 50us/sample - loss: 0.0461 - accuracy: 0.9946
Epoch 7/10
2210/2210 [==============================] - 0s 56us/sample - loss: 0.0090 - accuracy: 0.9986
Epoch 8/10


3094/3094 [==============================] - 0s 41us/sample - loss: 4.1668e-06 - accuracy: 1.0000
Epoch 4/10
3094/3094 [==============================] - 0s 40us/sample - loss: 3.7341e-06 - accuracy: 1.0000
Epoch 5/10
3094/3094 [==============================] - 0s 41us/sample - loss: 4.8849e-06 - accuracy: 1.0000
Epoch 6/10
3094/3094 [==============================] - 0s 44us/sample - loss: 6.1177e-06 - accuracy: 1.0000
Epoch 7/10
3094/3094 [==============================] - 0s 45us/sample - loss: 4.4616e-06 - accuracy: 1.0000
Epoch 8/10
3094/3094 [==============================] - 0s 46us/sample - loss: 3.6383e-06 - accuracy: 1.0000
Epoch 9/10
3094/3094 [==============================] - 0s 41us/sample - loss: 3.4777e-06 - accuracy: 1.0000
Epoch 10/10
3094/3094 [==============================] - 0s 43us/sample - loss: 3.6031e-06 - accuracy: 1.0000
predict shape: (100, 1)
Train on 3536 samples
Epoch 1/10
3536/3536 [==============================] - 0s 53us/sample - loss: 3.7668e-06 - 

3978/3978 [==============================] - 0s 45us/sample - loss: 2.3671e-05 - accuracy: 1.0000
Epoch 6/10
3978/3978 [==============================] - 0s 46us/sample - loss: 2.1753e-05 - accuracy: 1.0000
Epoch 7/10
3978/3978 [==============================] - 0s 55us/sample - loss: 2.0364e-05 - accuracy: 1.0000
Epoch 8/10
3978/3978 [==============================] - 0s 68us/sample - loss: 1.9106e-05 - accuracy: 1.0000
Epoch 9/10
3978/3978 [==============================] - 0s 59us/sample - loss: 1.8069e-05 - accuracy: 1.0000
Epoch 10/10
3978/3978 [==============================] - 0s 49us/sample - loss: 1.7154e-05 - accuracy: 1.0000
predict shape: (100, 1)
Train on 3978 samples
Epoch 1/10
3978/3978 [==============================] - 0s 89us/sample - loss: 1.6339e-05 - accuracy: 1.0000
Epoch 2/10
3978/3978 [==============================] - 0s 73us/sample - loss: 1.5611e-05 - accuracy: 1.0000
Epoch 3/10
3978/3978 [==============================] - 0s 79us/sample - loss: 1.4962e-05 - 

In [37]:
avg_prec

[0.9615384615384615,
 0.9506531204644413,
 0.912825469429243,
 0.9158249158249158,
 0.9259259259259259,
 0.9124579124579124,
 0.9259259259259259,
 0.9259259259259259,
 0.8998189919242551]

In [34]:
# a=[0.9259259259259259,
#  0.9259259259259259,
#  0.9259259259259259,
#  0.9259259259259259,
#  0.9225589225589225,
#  0.909090909090909,
#  0.9225589225589225,
#  0.9124579124579124,
#  0.933048433048433]